In [2]:
"""
!pip install git+https://github.com/huggingface/datasets.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install torchaudio
!pip install librosa
!pip install jiwer
"""

'\n!pip install git+https://github.com/huggingface/datasets.git\n!pip install git+https://github.com/huggingface/transformers.git\n!pip install torchaudio\n!pip install librosa\n!pip install jiwer\n'

In [7]:
from datasets import load_dataset
common_voice_train = load_dataset("mozilla-foundation/common_voice_11_0", "ta", trust_remote_code=True, split="train+validation")
common_voice_test = load_dataset("mozilla-foundation/common_voice_11_0", "ta", trust_remote_code=True, split="test")

In [8]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [9]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [6]:
#show_random_elements(common_voice_train.remove_columns(["path"]), num_examples=2)

,audio,sentence
0,"{'path': '/home/vmadmin/.cache/huggingface/datasets/downloads/extracted/6e584255450c6ef089111d1246a070c397022fd026f7bbcba10d9ca510b3f859/ta_train_0/common_voice_ta_26769796.mp3', 'array': [-9.094947017729282e-13, -6.366462912410498e-12, -1.1368683772161603e-11, -6.59383658785373e-12, 5.684341886080801e-13, 4.320099833421409e-12, 7.44648787076585e-12, 1.0231815394945443e-11, 9.43600753089413e-12, 1.2732925824820995e-11, 2.2509993868879974e-11, 2.887645678129047e-11, 2.637534635141492e-11, 2.546585164964199e-11, 2.546585164964199e-11, 2.3419488570652902e-11, 1.6541434888495132e-11, 6.480149750132114e-12, -1.887201506178826e-11, -4.4565240386873484e-11, -5.729816621169448e-11, -5.502442945726216e-11, -4.774847184307873e-11, -2.97859514830634e-11, -6.821210263296962e-13, 3.1604940886609256e-11, 6.616573955398053e-11, 9.617906471248716e-11, 1.0254552762489766e-10, 7.969447324285284e-11, 4.376943252282217e-11, -7.844391802791506e-12, -7.048583938740194e-11, -1.1300471669528633e-10, -1.340367816737853e-10, -1.552962203277275e-10, -1.7257661966141313e-10, -1.7598722479306161e-10, -1.8917489796876907e-10, -2.0554580260068178e-10, -1.950866135302931e-10, -1.6189005691558123e-10, -1.2232703738845885e-10, -5.88897819397971e-11, 3.126388037344441e-11, 1.297451035497943e-10, 2.2291146706265863e-10, 2.9831426218152046e-10, 3.4640379453776404e-10, 3.9113956518121995e-10, 4.4428816181607544e-10, 4.240519047016278e-10, 2.2589574655285105e-10, -1.0697931429604068e-10, -4.0313352656085044e-10, -5.118181434227154e-10, -4.3314685171935707e-10, -2.856950231944211e-10, -2.0907009457005188e-10, -2.6761881599668413e-10, -4.5292836148291826e-10, -7.039488991722465e-10, -9.613358997739851e-10, -1.203261490445584e-09, -1.4424585970118642e-09, -1.6680132830515504e-09, -1.7698766896501184e-09, -1.5606929082423449e-09, -9.467839845456183e-10, -1.20508047984913e-10, 5.536548997042701e-10, 9.113136911764741e-10, 1.0660414773155935e-09, 1.0907115211011842e-09, 8.146798791131005e-10, 4.9112713895738125e-11, -1.067974153556861e-09, -2.0513652998488396e-09, -2.5065673980861902e-09, -2.49178810918238e-09, -2.323986336705275e-09, -2.149590727640316e-09, -1.86355464393273e-09, -1.3956196198705584e-09, -8.803908713161945e-10, -5.456968210637569e-10, -5.566107574850321e-10, -9.831637726165354e-10, -1.680291461525485e-09, -2.1668711269740015e-09, -1.8062564777210355e-09, -3.878994903061539e-10, 1.4904344425303861e-09, 2.8869635571027175e-09, 3.365244083397556e-09, 3.2534899219172075e-09, 3.0595401767641306e-09, 2.8478552849264815e-09, 2.329329618078191e-09, 1.401076588081196e-09, ...], 'sampling_rate': 48000}",ஆயினும் அக்காலத்து மக்கள் எல்லோரும் பொதுவாக மூடர்கள்.
1,"{'path': '/home/vmadmin/.cache/huggingface/datasets/downloads/extracted/6e584255450c6ef089111d1246a070c397022fd026f7bbcba10d9ca510b3f859/ta_train_0/common_voice_ta_26036480.mp3', 'array': [2.842170943040401e-14, 1.2505552149377763e-12, 1.8758328224066645e-12, 1.4210854715202004e-12, 9.094947017729282e-13, 7.105427357601002e-13, 7.887024366937112e-13, 9.805489753489383e-13, 1.2221335055073723e-12, 1.4637180356658064e-12, 1.7692514120426495e-12, 1.5205614545266144e-12, 6.679101716144942e-13, 2.5579538487363607e-13, 8.526512829121202e-13, 8.242295734817162e-13, 3.694822225952521e-13, -1.7053025658242404e-13, -1.4779288903810084e-12, -2.9558577807620168e-12, -3.069544618483633e-12, -1.9468870959826745e-12, -1.0516032489249483e-12, -2.7711166694643907e-13, 1.7053025658242404e-13, 5.968558980384842e-13, 1.8474111129762605e-12, 3.858247055177344e-12, 5.357492227631155e-12, 5.5138116294983774e-12, 4.490630090003833e-12, 1.5063505998114124e-12, -1.9184653865522705e-12, -3.225864020350855e-12, -2.6432189770275727e-12, -2.2453150450019166e-12, -3.268496584496461e-12, -7.474909580196254e-12, -1.4424017535930034e-11, -1.9731771772057982e-11, -2.0548895918182097e-11, -1.757882728270488e-11, -1.3358203432289883e-11, -1.254818471352337e-11, -1.9028334463655483e-11, -3.015543370565865e-11, -3.9364067561109

In [10]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [11]:
common_voice_train = common_voice_train.map(remove_special_characters, remove_columns=["sentence"])
common_voice_test = common_voice_test.map(remove_special_characters, remove_columns=["sentence"])

In [12]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [13]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

Map:   0%|          | 0/53468 [00:00<?, ? examples/s]

Map:   0%|          | 0/53468 [00:00<?, ? examples/s]

In [14]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [15]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'¾': 0,
 'ஓ': 1,
 'ஏ': 2,
 '●': 3,
 'j': 4,
 'ை': 5,
 'b': 6,
 '◯': 7,
 'š': 8,
 'z': 9,
 'ோ': 10,
 'ஃ': 11,
 'h': 12,
 'i': 13,
 'x': 14,
 ')': 15,
 'ள': 16,
 'ர': 17,
 '—': 18,
 'v': 19,
 'ஐ': 20,
 'ி': 21,
 'ச': 22,
 'ஸ': 23,
 'ீ': 24,
 'm': 25,
 '_': 26,
 'l': 27,
 'ண': 28,
 'á': 29,
 'ô': 30,
 'வ': 31,
 'ஞ': 32,
 'ஆ': 33,
 'ு': 34,
 'ூ': 35,
 '…': 36,
 'ட': 37,
 'ஊ': 38,
 'ப': 39,
 'p': 40,
 'ன': 41,
 'ே': 42,
 'எ': 43,
 'n': 44,
 't': 45,
 'ஔ': 46,
 'ഥ': 47,
 'q': 48,
 'ந': 49,
 'ஷ': 50,
 'ா': 51,
 'அ': 52,
 'u': 53,
 '‚': 54,
 '•': 55,
 '’': 56,
 'é': 57,
 'c': 58,
 'ங': 59,
 'f': 60,
 'ௌ': 61,
 'ற': 62,
 'ல': 63,
 'r': 64,
 'o': 65,
 '\\': 66,
 's': 67,
 'k': 68,
 'உ': 69,
 'ஒ': 70,
 'e': 71,
 '&': 72,
 '(': 73,
 'ொ': 74,
 'ம': 75,
 'd': 76,
 '்': 77,
 'w': 78,
 'g': 79,
 'ழ': 80,
 'த': 81,
 '`': 82,
 'இ': 83,
 'ஈ': 84,
 'a': 85,
 'ெ': 86,
 'y': 87,
 'ௗ': 88,
 ' ': 89,
 'ஹ': 90,
 '–': 91,
 'க': 92,
 'ஜ': 93,
 '·': 94,
 'ய': 95,
 "'": 96}

In [16]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [17]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

99

In [18]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [19]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

/home/vmadmin/venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [20]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [21]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [22]:
 processor.save_pretrained("wav2vec2-large-xlsr-tamil-demo")

[]

In [23]:
import torchaudio

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["text"]
    return batch

In [24]:
common_voice_train = common_voice_train.map(speech_file_to_array_fn, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(speech_file_to_array_fn, remove_columns=common_voice_test.column_names)


In [25]:
import librosa
import numpy as np
import librosa
import numpy as np

def resample(batch):
    speech_array = np.asarray(batch["speech"])
    batch["speech"] = librosa.resample(speech_array, orig_sr=48_000, target_sr=16_000)
    batch["sampling_rate"] = 16_000
    return batch



In [26]:
common_voice_train = common_voice_train.map(resample, num_proc=4)
common_voice_test = common_voice_test.map(resample, num_proc=4)

In [27]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [28]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names, batch_size=8, num_proc=4, batched=True)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names, batch_size=8, num_proc=4, batched=True)

Map (num_proc=4):   0%|          | 0/53468 [00:00<?, ? examples/s]

/home/vmadmin/venv/lib/python3.11/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/vmadmin/venv/lib/python3.11/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/vmadmin/venv/lib/python3.11/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text

Map (num_proc=4):   0%|          | 0/11815 [00:00<?, ? examples/s]

/home/vmadmin/venv/lib/python3.11/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/vmadmin/venv/lib/python3.11/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/vmadmin/venv/lib/python3.11/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text

In [29]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [30]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [31]:
import evaluate
wer_metric = evaluate.load("wer")
"""
from datasets import load_metric
wer_metric = load_metric("wer")
"""

'\nfrom datasets import load_metric\nwer_metric = load_metric("wer")\n'

In [30]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 996.5 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [32]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [33]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
model.freeze_feature_extractor()

/home/vmadmin/venv/lib/python3.11/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:2177: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [35]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  #output_dir="/content/gdrive/MyDrive/wav2vec2-large-xlsr-tamil-demo",
  output_dir="./wav2vec2-large-xlsr-Tamil-demo",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

/home/vmadmin/venv/lib/python3.11/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [36]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

SyntaxError: invalid syntax (3066402069.py, line 1)

In [ ]:
trainer.train()

/home/vmadmin/venv/lib/python3.11/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/vmadmin/venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss
